In [3]:
import mirdata
print(mirdata.list_datasets())
maestro = mirdata.initialize('maestro', data_home='/root/musica/data')

['acousticbrainz_genre', 'baf', 'ballroom', 'beatles', 'beatport_key', 'billboard', 'brid', 'candombe', 'cante100', 'cipi', 'compmusic_carnatic_rhythm', 'compmusic_carnatic_varnam', 'compmusic_hindustani_rhythm', 'compmusic_indian_tonic', 'compmusic_jingju_acappella', 'compmusic_otmm_makam', 'compmusic_raga', 'cuidado', 'da_tacos', 'dagstuhl_choirset', 'dali', 'egfxset', 'filosax', 'four_way_tabla', 'freesound_one_shot_percussive_sounds', 'giantsteps_key', 'giantsteps_tempo', 'good_sounds', 'groove_midi', 'gtzan_genre', 'guitarset', 'hainsworth', 'haydn_op20', 'idmt_smt_audio_effects', 'ikala', 'irmas', 'maestro', 'mdb_stem_synth', 'medley_solos_db', 'medleydb_melody', 'medleydb_pitch', 'mridangam_stroke', 'mtg_jamendo_autotagging_moodtheme', 'openmic2018', 'orchset', 'phenicx_anechoic', 'queen', 'rwc_classical', 'rwc_jazz', 'rwc_popular', 'salami', 'saraga_carnatic', 'saraga_hindustani', 'scms', 'simac', 'slakh', 'tinysol', 'tonality_classicaldb', 'tonas', 'vocadito']


INFO: Note: NumExpr detected 64 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.
INFO: NumExpr defaulting to 16 threads.


In [ ]:
maestro.download()

INFO: Downloading ['all'] to /root/musica/data
INFO: [all] downloading maestro-v2.0.0.zip
 40%|████      | 41.2G/102G [19:55<51:51, 21.1MB/s]      